In [100]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
n_inputs = 13
n_hidden = 20
n_classes = 3

num_samples = 178

In [102]:
def logits_function(p, X):
    # Roll-back the weights and biases
    W1 = p[0:260].reshape((n_inputs,n_hidden))
    b1 = p[260:280].reshape((n_hidden,))
    W2 = p[280:340].reshape((n_hidden,n_classes))
    b2 = p[340:343].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [103]:
# Forward propagation
def forward_prop(params, X,y):
    logits = logits_function(params, X)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood
    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [104]:
def Get_PSO(f):
    # Initialize swarm
    options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

    # Call instance of PSO
    dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
    optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=1000)
    return pos

In [105]:
def predict(pos, X):
    logits = logits_function(pos, X)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [106]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

labelencoder = LabelEncoder()

def Start_Wine():
    data = pd.read_csv('databases/wine.data', names = ['class', 'alcohol', 'malic acid', 'ash', 'alcalinity of ash', 'magnesium', 'total phenols', 'flavanoids', 'nonflavanoid phenols', 'proanthocyanins', 'color intensity', 'hue', 'diluted', 'proline'])

    target = labelencoder.fit_transform(data['class'].values)
    data_drop = data.drop('class',axis=1)
    data = data_drop.values

    sc = StandardScaler()
    data = sc.fit_transform(data)

    def f(x):
        n_particles = x.shape[0]
        j = [forward_prop(x[i], data, target) for i in range(n_particles)]
        return np.array(j)

    pos = Get_PSO(f)
    scores = (predict(pos, data) == target)

    print("\n Wine \n")
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())

In [107]:
Start_Wine()

2022-03-07 20:18:04,988 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=1.3e-6 
2022-03-07 20:18:33,851 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.301376175706591e-06, best pos: [ 2.76680165e+00 -3.43426133e-01 -1.62491927e+00 -1.00982217e-02
  1.43465143e+00  2.61460629e-01 -2.67233506e-01 -2.80833040e-01
  1.38502761e+00  2.15152571e-01  1.84412251e-01  5.81682215e-01
  4.26704486e-01 -3.91892867e-02  2.35240617e-01  1.00814125e-01
 -1.78476400e+00  2.00460264e+00  1.27653205e+00 -3.11202732e-01
  2.78169677e-01 -3.09333380e-01 -8.40032215e-02 -9.39736429e-01
 -9.43559267e-01  1.54789191e+00 -6.60405369e-02 -2.50104098e-01
  1.00476577e+00 -3.72420101e-02  1.81440641e+00  7.38752663e-01
 -2.73437600e-01 -5.25076519e-01  3.40565732e-01 -5.91069263e+00
  3.53478540e-01 -1.00949531e+00  1.05607023e+00  2.04863404e-01
 -9.00735554e-02 -6.


 Wine 

Mean:  1.0
Standard Deviation:  0.0
